# 数据载入、存储及文件格式

In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [105]:
df = pd.read_csv("./examples/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [106]:
# 可以使用read_table，并指定分隔符
df1 = pd.read_table("./examples/ex1.csv", sep=',')
df1

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [107]:
# 有的文件不包含表头
df2 = pd.read_csv("./examples/ex2.csv", header=None)
df2

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [108]:
# 也可以手动指定表头
df3 = pd.read_csv("./examples/ex2.csv", names=['a', 'b', 'c', 'd', 'message'])
df3

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [109]:
!cat./ examples / csv_mindex.csv

zsh:1: no such file or directory: cat./


In [110]:
# 当想要从多个列中形成一个分层索引，需要传入一个包含列序号或列名的列表
df4 = pd.read_csv("./examples/csv_mindex.csv", index_col=['key1', 'key2'])
df4

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [111]:
!cat./ examples / ex3.txt

zsh:1: no such file or directory: cat./


In [112]:
list(open("./examples/ex3.txt"))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [113]:
# 当分隔符由不同数量的空格分隔开时，可以使用正则表达式来处理
# \s+：至少匹配一个空格
result = pd.read_table('./examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [114]:
!cat./ examples / ex4.csv

zsh:1: no such file or directory: cat./


In [115]:
# 可以跳过指定的行
pd.read_csv('./examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [116]:
# 缺失值处理
!cat./ examples / ex5.csv

zsh:1: no such file or directory: cat./


In [117]:
result1 = pd.read_csv("./examples/ex5.csv", na_values=['NULL'])
result1

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [118]:
# 在字典中，每列可以指定不同缺失值标识
# 将message中的foo和something中的two置为NA
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('./examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [119]:
# 设置pandas设置显示行数
pd.options.display.max_rows = 10

In [120]:
# 只读取前5行
result = pd.read_csv("./examples/ex6.csv", nrows=5)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [121]:
# 为了分块读入文件，可以指定chunksize作为每一块的行数
chunker = pd.read_csv("./examples/ex6.csv", chunksize=1000)
chunker

In [122]:
tot = pd.Series([], dtype='float64')
# 按照chunksize的大小来遍历目标文件，并对key列聚合获得技术值
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)

In [123]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### 将数据写入文件格式

In [124]:
data = pd.read_csv("./examples/ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [125]:
data.to_csv("./output/out.csv")

In [126]:
# 也可以指定其他分隔符
import sys

data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [127]:
# 将空值输出为NULL
data.to_csv(sys.stdout, na_rep='NULL', sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|NULL
1|two|5|6|NULL|8|world
2|three|9|10|11.0|12|foo


In [128]:
# header和index不输出
data.to_csv(sys.stdout, sep="|", header=False, index=False)

one|1|2|3.0|4|
two|5|6||8|world
three|9|10|11.0|12|foo


In [129]:
# Series也可以写入csv
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv(sys.stdout, sep="|")

|0
2000-01-01|0
2000-01-02|1
2000-01-03|2
2000-01-04|3
2000-01-05|4
2000-01-06|5
2000-01-07|6


### 使用分隔格式

In [130]:
!cat./ examples / ex7.csv

zsh:1: no such file or directory: cat./


In [131]:
# 对于任何带有单字符分隔符的文件，可以使用Python的内建csv模块。
# 要使用它，需要将任一打开的文件或文件型对象传给csv.reader
import csv

f = open("./examples/ex7.csv")
reader = csv.reader(f)
for line in reader:
    print(line)
f.close()

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [132]:
with open("./examples/ex7.csv") as f:
    lines = list(csv.reader(f))

In [133]:
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [134]:
header, values = lines[0], lines[1:]

In [135]:
header

['a', 'b', 'c']

In [136]:
values

[['1', '2', '3'], ['1', '2', '3']]

In [137]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [138]:
# 字典转化为DataFrame
df = pd.DataFrame(data_dict)
df

,a,b,c
0,1,2,3
1,1,2,3


In [139]:
# 当CSV文件有不同的风格时，可以根据不同的分隔符、字符串引用约定或行终止的一种新的格式时，
# 可以使用csv.Dialect定义一个简单的子类
class my_dialect(csv.Dialect):
    lineterminator = '\n'  #行终止符
    delimiter = ';'  # 分隔符
    quotechar = '"'  # 用在特殊字符字段中的引号
    quoting = csv.QUOTE_MINIMAL  # 引用惯例，默认为csv.QUOTE_MINIMAL：是只使用特殊字符，如分隔符


f = open("./examples/ex7.csv")
reader = csv.reader(f, dialect=my_dialect)
for line in reader:
    print(line)

['a,"b","c"']
['1,"2","3"']
['1,"2","3"']


In [140]:
# 也可以不定义子类，直接使用分隔符关键字
reader = csv.reader(f, delimiter='|')
f.close()

In [141]:
# 写入csv文件
with open("./examples/mydata.csv", 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow((1, 2, 3))
    writer.writerow((4, 5, 6))
    writer.writerow((7, 8, 9))

### JSON数据

In [142]:
# 使用json加载JSON数据
import json

In [143]:
obj = """
{"name": "Wes",
 "places_lived":["United States", "Spain", "Germany"],
 "pet":null,
 "siblings": [{"name": "Scott", "age":30, "pets":["Zeus","Zuko"]},
              {"name": "Katie", "age":38, "pets":["Sixes","Stache","Cisco"]}]
}
"""

In [144]:
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [145]:
type(result)

dict

In [146]:
# 使用dumps将对象转回JSON
asjson = json.dumps(result)
type(asjson)

str

In [147]:
# 将字典构成的列表传入DataFrame，并选出数据字段的子集
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age', 'pets'])

In [148]:
siblings

,name,age,pets
0,Scott,30,"[Zeus, Zuko]"
1,Katie,38,"[Sixes, Stache, Cisco]"


In [149]:
# pandas.read_json可以自动将JSON数据集按照指定次序转换为Series或DataFrame
!cat./ examples / example.json

zsh:1: no such file or directory: cat./


In [150]:
data = pd.read_json("./examples/example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [151]:
# 从pandas中将数据导出为json
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [152]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### HTML和XML读取

In [153]:
import lxml

In [154]:
# 使用read_html解析<table>标签，返回为DataFrame对象的列表
tables = pd.read_html("./examples/fdic_failed_bank_list.html")
len(tables)

1

In [155]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [156]:
type(failures)

pandas.core.frame.DataFrame

In [157]:
# 简单的数据清理和分析工作
# 计算每年银行倒闭的数量
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.head()

0   2016-09-23
1   2016-08-19
2   2016-05-06
3   2016-04-29
4   2016-03-11
Name: Closing Date, dtype: datetime64[ns]

In [158]:
type(close_timestamps)

pandas.core.series.Series

In [159]:
close_timestamps.dt.year.value_counts().head()

2010    157
2009    140
2011     92
2012     51
2008     25
Name: Closing Date, dtype: int64

## 二进制格式

In [160]:
# pickle序列化模块进行二进制格式操作
# pandas对西那个拥有一个to_pickle方法可以将数据以pickle格式写入硬盘
frame = pd.read_csv("./examples/ex1.csv")
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [161]:
frame.to_pickle("./examples/frame_pickle")

In [162]:
# 使用内建对象读取"pickle化"的文件
pd.read_pickle("./examples/frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


> pickle仅被推荐为短期的存储格式，问题在于pickle很难确保格式的长期有效性.因为后期版本很难保证当前的pickle格式能被正确序列化

### 使用HDF5

HDF5中的"HDF"代表分层数据格式。每个HDF5文件可以存储多个数据集并且支持元数据。与更简单的格式相比，HDF5支持多种压缩模式，使得重复模式的数据可以更加高效地存储。HDF5适用于处理不适合在内存中存储的超大型数据，可以高效读写大型数据中的一小块。

In [163]:
# pandas的HDFStore类可以像字典一样工作并处理低级别细节
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']

In [164]:
# 包含在HDF5文件中的对象可以使用相同的字典型API进行检索
store['obj1'].head()

,a
0,-0.846989
1,-0.589804
2,0.364022
3,-1.167785
4,-1.331951


In [165]:
# HDFStore支持两种存储模式，'fixed'和'table'，后者速度较慢，但支持一种特殊操作
"""
``'fixed'``
                Fixed format.  Fast writing/reading. Not-appendable, nor searchable.
``'table'``
                Table format.  Write as a PyTables Table structure which may perform
                worse but allow more flexible operations like searching / selecting
                subsets of the data.
"""
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])
store.close()

In [166]:
# put是store['obj2]=frame方法的显示版本，但允许设置其他选项，如存储格式
frame.to_hdf('mydata1.h5', 'obj3', format='table')
pd.read_hdf('mydata1.h5', 'obj3', where=['index < 5'])

,a
0,-0.846989
1,-0.589804
2,0.364022
3,-1.167785
4,-1.331951


HDF5不是数据库，是一种适合一次写入多次读取的数据集。尽管数据可以在任何时间加到文件中，但如果多个写入着持续写入，文件可能损坏。

### 读取Excel文件

In [167]:
xlsx = pd.ExcelFile("./examples/ex1.xlsx")
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [168]:
# 在读取时，可以指定读取的Sheet页，加快文件读取速度
frame = pd.read_excel('./examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [169]:
# 如果需要将pandas数据写入到Excel中，首先要生成一个ExcelWriter，然后使用pandas对象的to_excel方法将数据写入
writer = pd.ExcelWriter('./examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [170]:
# 也可以将文件路径传给to_excel，避免直接调用ExcelWriter
frame.to_excel('./examples/ex3.xlsx')

## 与Web API交互

In [172]:
import requests

url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [174]:
# data中的每一个元素都是一个包含Githup问题页面上的所有数据的字典（注释除外），可以将数据data直接传给DataFrame
data = resp.json()
data[0]['title']

'PERF: Improve SeriesGroupBy.value_counts performances with categorical values.'

In [175]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues

,number,title,labels,state
0,46940,PERF: Improve SeriesGroupBy.value_counts perfo...,[],open
1,46939,BUG: pd.wide_to_long can't handle stubname equ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
2,46938,ENH: validates boolean kwargs in DataFrame and...,[],open
3,46937,TYP: towards reportUnusedImport,[],open
4,46936,Add/reorganize scalar Timedelta tests,[],open
...,...,...,...,...
25,46896,BUG: incorrect type when casting to nullable t...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
26,46895,Leading zeros in data for read_excel vs read_csv,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
27,46894,PERF: Optimize read_excel nrows,"[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1M...",open
28,46892,BUG: rolling aggregate() with a list of functi...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open


## 与数据库交互

In [178]:
# 使用Python内建的sqlite3驱动来生成一个SQLite数据库
import sqlite3

delete = """drop table if exists test;"""
query = """
create table test
(a varchar(20),
 b varchar(20),
 c real,
 d integer
);
"""
con = sqlite3.connect('mydata.sqlite')
con.execute(delete)
con.execute(query)
con.commit()

In [179]:
# 插入数据
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "insert into test values(?,?,?,?)"
con.executemany(stmt, data)
con.commit()

In [180]:
# 查询数据
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [181]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [182]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
